In [128]:
from google.cloud import bigquery

In [236]:
import pandas as pd
path = '/Users/bernardoaraujo/code/bchojniak/filmaholic/raw_data/ml-25m'
movies_df = pd.read_csv(path + '/' + 'movies.csv',dtype='str')
ratings_df = pd.read_csv(path + '/' + 'ratings.csv',dtype='str').iloc[:500000, :]
tags_df = pd.read_csv(path + '/' + 'tags.csv',dtype='str')
links_df = pd.read_csv(path + '/' + 'links.csv',dtype='str')
genome_tags_df = pd.read_csv(path + '/' + 'genome-tags.csv',dtype='str')
genome_scores_df = pd.read_csv(path + '/' + 'genome-scores.csv',dtype='str')


In [155]:
len(links_df)

62423

In [76]:
imdb_id = links_df['imdbId'].sample()
imdb_id = imdb_id.values[0]
url = (f'https://www.imdb.com/title/tt{imdb_id}')
print(url)

https://www.imdb.com/title/tt3799996


In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

response = requests.get(url,headers=HEADERS)
soup = BeautifulSoup(response.content, "html.parser")


In [78]:
# sample_size = 10

# def generate_random_imdb_id():
#     return str(links_df['imdbId'].sample().values[0])

# def imdb_list(sample_size):
#     imdb_id_list = [generate_random_imdb_id() for _ in range(sample_size)]
#     return imdb_id_list
# imdb_list(sample_size)

['0020876',
 '5364518',
 '2978716',
 '1277146',
 '1794801',
 '4691804',
 '0360130',
 '0056713',
 '7429362',
 '0057372']

# Features Scraped

## Release Date

In [79]:
def date_search(imdb_id):
    url = (f'https://www.imdb.com/title/tt{imdb_id}')
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    
    search_list = soup.find_all("a", class_ = "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    
    for date in search_list:
        if 'releaseinfo' in date['href']:
            return date.text
date_search(imdb_id)

'November 27, 2014 (Australia)'

## Country of Origin

In [80]:
def country_search(imdb_id):
    url = (f'https://www.imdb.com/title/tt{imdb_id}')
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    
    search_list = soup.find_all("a", class_ = "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    
    for country in search_list:
        if 'country_of_origin' in country['href']:
            return country.text
country_search(imdb_id)

'Australia'

## Movie Duration

In [81]:
# imdb_id = links_df['imdbId'].sample()
# imdb_id = imdb_id.values[0]
# duration_url = (f'https://www.imdb.com/title/tt{imdb_id}/technical')

# response = requests.get(duration_url,headers=HEADERS)
# duration_soup = BeautifulSoup(response.content, "html.parser")

'''Have a better look at this'''

'Have a better look at this'

In [82]:
def movie_duration(imdb_id):    
    url = (f'https://www.imdb.com/title/tt{imdb_id}')
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    
    search_duration = soup.find_all("li", class_ = "ipc-metadata-list__item")
    
    for search in search_duration:
        if 'data-testid' in search.attrs and 'title-techspec_runtime' in search['data-testid']:
            duration = search.text.strip()
            duration = duration.replace("Runtime", "").strip()
            return duration
movie_duration(imdb_id)

'1 hour 39 minutes'

## Director

In [83]:
crew_url = (f'https://www.imdb.com/title/tt{imdb_id}/fullcredits')

response1 = requests.get(crew_url,headers=HEADERS)
crew_soup = BeautifulSoup(response1.content, "html.parser")

print(crew_url)

https://www.imdb.com/title/tt3799996/fullcredits


In [84]:
def director_search(imdb_id):
    url = f'https://www.imdb.com/title/tt{imdb_id}/fullcredits'
    response = requests.get(url, headers=HEADERS)
    crew_soup = BeautifulSoup(response.content, "html.parser")
    
    table_direct = crew_soup.find("table", class_ = "simpleTable simpleCreditsTable")
    
    for table in range(len(table_direct)):
        if table == 1:
            director = crew_soup.find("td", class_ = "name")
            director = director.text.replace("\n","")
            return director

director_search(imdb_id)

' Paul Fenech '

## Actors

In [113]:
def actors_search(imdb_id):
    crew_url = f'https://www.imdb.com/title/tt{imdb_id}/fullcredits'
    response = requests.get(crew_url, headers=HEADERS)
    crew_soup = BeautifulSoup(response.content, "html.parser")
    
    table_act = crew_soup.find("table", class_="cast_list")
    
    mt = []
    for ppl in table_act:
        ppl = ppl.text.replace("\n", "").strip()
        mt.append(ppl)
    
    actors = []
    for actor in mt:
        actor_name = ""
        for c in range(len(actor)):
            if actor[c] == '.':
                actor_name = actor[:c].strip()
                break
        if actor_name:
            actors.append(actor_name)
    actors = actors[:3]
    return actors
actors_search(imdb_id)

['Paul Fenech', 'Elle Dawe', 'Kevin Taumata']

## Music by

In [114]:
table_music = crew_soup.find_all("table", class_ = "simpleTable simpleCreditsTable")

# for music in table_music:
#     print(music.text)
#     print("\n")
    
# for music in range(len(table_music)):
#     if music == 2:
#         musician = crew_soup.find("td", class_ = "name")
#         print(musician.text)


# Function

In [156]:
%%time
import requests
from bs4 import BeautifulSoup
import pandas as pd

sample_size = 1000

def generate_random_imdb_id():
    return str(links_df['imdbId'].sample().values[0])


def imdb_list(sample_size):
    imdb_id_list = [generate_random_imdb_id() for _ in range(sample_size)]
    return imdb_id_list

def features_scraped():
    id_list = imdb_list(sample_size)
    print(id_list)
    
    data_list = []
    error_list = []
    working_id_list = []
    for id in id_list:
        try:
            release = date_search(id)
            origin = country_search(id)
            movie_length = movie_duration(id)
            director_name = director_search(id)
            actors_names = actors_search(id)
            actor_1,actor_2,actor_3 = actors_names[0],actors_names[1],actors_names[2]
            
            data_list.append([release, origin, movie_length, director_name, actor_1,actor_2,actor_3])
            
            working_id_list.append(id)
        except:
            print(id)
            error_list.append(id)
            
    print(working_id_list)   
    columns = ['Release_Date', 'Country_of_Origin', 'Movie_Length', 'Director', 'Actor_1','Actor_2','Actor_3']
    scraped_df = pd.DataFrame(data=data_list, columns=columns, index =working_id_list)
    scraped_df = scraped_df.reset_index().rename(columns = {'index':'IMDBid'})
        
    return (scraped_df, error_list)

result, errors = features_scraped()
result

['0120381', '5208950', '2364841', '2854926', '4397382', '0041788', '3917908', '0049470', '7460806', '1555803', '0079702', '0477302', '4240654', '3563782', '0088170', '5526028', '1685576', '0109934', '0257044', '0092809', '2980794', '0436078', '1155053', '0068327', '1073131', '7319496', '2641152', '0397363', '0048248', '8305116', '1464251', '0119517', '2130282', '4920274', '0076119', '0806006', '0086148', '9308454', '0031442', '0092632', '0116046', '0037335', '3521442', '2343707', '0211359', '0019992', '0104509', '0098353', '0364930', '1830792', '0103003', '6762430', '0397078', '0117395', '0197857', '0067333', '0082602', '0078865', '0040587', '1865364', '0058025', '0338556', '0047435', '1978567', '0243278', '0377701', '0093215', '0365677', '1323973', '0079553', '0059014', '0407936', '0062271', '0051136', '0383694', '1846783', '0120744', '0871426', '0306016', '0352696', '0095993', '7128732', '1591622', '2746176', '0373889', '1282024', '0116992', '2980684', '1667353', '0104030', '2008633'

3563782
2641152
3521442
0117395
0040587
0058025
4535788
0343112
0284061
0294940
1172995
0156770
1830238
0284139
0358947
2765306
7923170
0059741
0449181
0249691
3347994
2128693
0171587
0302274
1378702
1798181
0666194
0073841
0953458
0199013
0881909
0026748
3228302
3528498
0224317
2065015
0145394
1587807
3103318
0975673
4991112
1205541
5652620
0046908
0315110
4057438
0000565
3823690
0103767
0273646
1367454
1235535
4835636
0109001
0104361
1500761
1740545
1756640
2464486
1320378
7410684
3569014
0056060
4363984
7026370
0223509
2484460
6086096
0037569
1653827
['0120381', '5208950', '2364841', '2854926', '4397382', '0041788', '3917908', '0049470', '7460806', '1555803', '0079702', '0477302', '4240654', '0088170', '5526028', '1685576', '0109934', '0257044', '0092809', '2980794', '0436078', '1155053', '0068327', '1073131', '7319496', '0397363', '0048248', '8305116', '1464251', '0119517', '2130282', '4920274', '0076119', '0806006', '0086148', '9308454', '0031442', '0092632', '0116046', '0037335',

,IMDBid,Release_Date,Country_of_Origin,Movie_Length,Director,Actor_1,Actor_2,Actor_3
0,0120381,"February 17, 1999 (Argentina)",United States,1 hour 32 minutes,Catherine Cyran,Kirsten Dunst,Zachery Ty Bryan,August Schellenberg
1,5208950,"January 26, 2018 (Poland)",United States,1 hour 50 minutes,Dimitri Logothetis,Alain Moussi,Sara Malakul Lane,Maxime Savaria
2,2364841,"October 4, 2013 (Brazil)",United States,1 hour 28 minutes,Brad Furman,Justin Timberlake,Ben Affleck,Gemma Arterton
3,2854926,"August 23, 2018 (Brazil)",United States,1 hour 40 minutes,Jeff Tomsic,Ed Helms,Lil Rel Howery,Jon Hamm
4,4397382,"October 16, 2015 (Sweden)",Sweden,1 hour 11 minutes,Pontus Klänge,Amanda Pajus,Lukas Holgersson,Tomas Norström
...,...,...,...,...,...,...,...,...
925,0310851,2002 (Canada),Canada,1 hour 30 minutes,Rodney Gibbons,Matt Frewer,Kenneth Welsh,Shawn Lawrence
926,1427241,"May 23, 2009 (Japan)",Japan,1 hour 58 minutes,Jun'ichi Mori,Ryo Kase,Masaki Okada,Fumiyo Kohinata
927,0110893,"June 1, 1994 (Czech Republic)",Czech Republic,1 hour 39 minutes,Zdenek Troska,Radek Valenta,Andrea Cerná,Yvetta Blanarovicová
928,0142239,"March 12, 1994 (Japan)",Japan,48 minutes,Shigeyasu Yamauchi,Masako Nozawa,Mayumi Tanaka,Takeshi Kusao


In [157]:
result

,IMDBid,Release_Date,Country_of_Origin,Movie_Length,Director,Actor_1,Actor_2,Actor_3
0,0120381,"February 17, 1999 (Argentina)",United States,1 hour 32 minutes,Catherine Cyran,Kirsten Dunst,Zachery Ty Bryan,August Schellenberg
1,5208950,"January 26, 2018 (Poland)",United States,1 hour 50 minutes,Dimitri Logothetis,Alain Moussi,Sara Malakul Lane,Maxime Savaria
2,2364841,"October 4, 2013 (Brazil)",United States,1 hour 28 minutes,Brad Furman,Justin Timberlake,Ben Affleck,Gemma Arterton
3,2854926,"August 23, 2018 (Brazil)",United States,1 hour 40 minutes,Jeff Tomsic,Ed Helms,Lil Rel Howery,Jon Hamm
4,4397382,"October 16, 2015 (Sweden)",Sweden,1 hour 11 minutes,Pontus Klänge,Amanda Pajus,Lukas Holgersson,Tomas Norström
...,...,...,...,...,...,...,...,...
925,0310851,2002 (Canada),Canada,1 hour 30 minutes,Rodney Gibbons,Matt Frewer,Kenneth Welsh,Shawn Lawrence
926,1427241,"May 23, 2009 (Japan)",Japan,1 hour 58 minutes,Jun'ichi Mori,Ryo Kase,Masaki Okada,Fumiyo Kohinata
927,0110893,"June 1, 1994 (Czech Republic)",Czech Republic,1 hour 39 minutes,Zdenek Troska,Radek Valenta,Andrea Cerná,Yvetta Blanarovicová
928,0142239,"March 12, 1994 (Japan)",Japan,48 minutes,Shigeyasu Yamauchi,Masako Nozawa,Mayumi Tanaka,Takeshi Kusao


In [158]:
PROJECT = "filmaholic-398017"
DATASET = "filmaholic"
TABLES = "scraped-features"

t = f"{PROJECT}.{DATASET}.{TABLES}"
    
client = bigquery.Client()
    
write_mode = "WRITE_TRUNCATE"
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
    
job = client.load_table_from_dataframe(result, t, job_config=job_config)
upload_scraped = job.result()

In [168]:
result.Director.head()

0        Catherine Cyran 
1     Dimitri Logothetis 
2            Brad Furman 
3            Jeff Tomsic 
4          Pontus Klänge 
Name: Director, dtype: object

In [227]:
directors_list = list(set(result.Director))

vectorized_directors = {}
vector_counter = 0
    
for director in directors_list:
    vectorized_directors[director] = vector_counter
    vector_counter += 1
    
print(vectorized_directors)

{' Roger Corman ': 0, ' Aleksandr Itygilov ': 1, ' Ugo Liberatore ': 2, ' Andrzej Bartkowiak ': 3, ' Robert Dornhelm ': 4, ' Drew Hall ': 5, ' Till Schauder ': 6, ' Doug Pray ': 7, ' Boaz Davidson ': 8, ' Johannes Wahlström ': 9, ' Barbara Wong Chun-Chun ': 10, ' Charles Barton ': 11, ' Yôjirô Takita ': 12, ' Stanley Kramer ': 13, ' Steve Loter ': 14, ' Robert Gaffney ': 15, ' Gianfranco Parolini ': 16, ' Jirô Nagae ': 17, ' José María Zabalza ': 18, ' Matt Chung-tien Wu ': 19, ' Adrián Cardona ': 20, ' Kalidas ': 21, ' Jackie Kong ': 22, ' Tassos Boulmetis ': 23, ' Nancy Criss ': 24, ' John Trent ': 25, ' Donald Cammell ': 26, ' Tim Robbins ': 27, ' Kheiron ': 28, ' Zubin Sarosh ': 29, ' Louis Morneau ': 30, ' Henry Levin ': 31, ' Gavin Hood ': 32, ' Madonna ': 33, ' Reshef Levi ': 34, ' Edward Buzzell ': 35, ' Sabine Bernardi ': 36, ' André Hunebelle ': 37, ' John Hamburg ': 38, ' Scott Derrickson ': 39, ' Geoff Burrowes ': 40, ' Masato Harada ': 41, ' Iain Morris ': 42, ' Terry O. M

In [228]:
Actors = pd.concat([result.Actor_1,result.Actor_2,result.Actor_3],ignore_index = True)
Actors_list = list(set(Actors))

vector_counter = 0
vectorized_actors = {}

for actor in Actors_list:
    vectorized_actors[actor] = vector_counter
    vector_counter += 1
    
print(len(vectorized_actors))

2586


In [229]:
result

,IMDBid,Release_Date,Country_of_Origin,Movie_Length,Director,Actor_1,Actor_2,Actor_3
0,0120381,"February 17, 1999 (Argentina)",United States,1 hour 32 minutes,Catherine Cyran,Kirsten Dunst,Zachery Ty Bryan,August Schellenberg
1,5208950,"January 26, 2018 (Poland)",United States,1 hour 50 minutes,Dimitri Logothetis,Alain Moussi,Sara Malakul Lane,Maxime Savaria
2,2364841,"October 4, 2013 (Brazil)",United States,1 hour 28 minutes,Brad Furman,Justin Timberlake,Ben Affleck,Gemma Arterton
3,2854926,"August 23, 2018 (Brazil)",United States,1 hour 40 minutes,Jeff Tomsic,Ed Helms,Lil Rel Howery,Jon Hamm
4,4397382,"October 16, 2015 (Sweden)",Sweden,1 hour 11 minutes,Pontus Klänge,Amanda Pajus,Lukas Holgersson,Tomas Norström
...,...,...,...,...,...,...,...,...
925,0310851,2002 (Canada),Canada,1 hour 30 minutes,Rodney Gibbons,Matt Frewer,Kenneth Welsh,Shawn Lawrence
926,1427241,"May 23, 2009 (Japan)",Japan,1 hour 58 minutes,Jun'ichi Mori,Ryo Kase,Masaki Okada,Fumiyo Kohinata
927,0110893,"June 1, 1994 (Czech Republic)",Czech Republic,1 hour 39 minutes,Zdenek Troska,Radek Valenta,Andrea Cerná,Yvetta Blanarovicová
928,0142239,"March 12, 1994 (Japan)",Japan,48 minutes,Shigeyasu Yamauchi,Masako Nozawa,Mayumi Tanaka,Takeshi Kusao


In [231]:
countries_list = list(set(result.Country_of_Origin))

vector_counter = 0
vectorized_countries = {}

for country in countries_list:
    vectorized_countries[country] = vector_counter
    vector_counter += 1
    
print(len(vectorized_countries))

54


In [237]:
ratings_df = pd.read_csv(path + '/' + 'ratings.csv',dtype='str')
ratings_df

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


# Loading Data

In [ ]:
PROJECT = "filmaholic-398017"
DATASET = "filmaholic"
cache_path = Path('raw_data/')

In [ ]:
tables_to_query = ['actors', 'directors', 'country', 'ratings', 'like_dislike_tags','movies']
dfs = {}

In [ ]:
for table in tables_to_query:

    file_cache_path = cache_path.joinpath(f"{table}.csv")

    if file_cache_path.is_file():
        dfs[table] = pd.read_csv(file_cache_path, dtype='str')

        print(f"Table {table} loaded from cache")

    else:
        pass 

In [ ]:
selected_ids = list(dfs['like_dislike_tags'].userId)

In [ ]:
merged = dfs['ratings'].merge(dfs['movies'], on='movieId', how='left')

# Processing

## Directors

In [ ]:
raw_directors = dfs['directors'].copy()
# raw_tags.userId = raw_tags.userId.astype('string')
# raw_tags.movieId = raw_tags.movieId.astype('string')
directors = raw_directors[raw_directors.userId.isin(user_list)]
directors = directors.drop('timestamp', axis=1).dropna()

In [ ]:
raw_movies = dfs['movies'].copy()
# raw_movies.movieId = raw_movies.movieId.astype('string')
movies = raw_movies[raw_movies.movieId.isin(movies_list)]

In [ ]:
raw_ratings = dfs['ratings'].copy()
# raw_ratings.movieId = raw_ratings.movieId.astype('string')
# raw_ratings.rating = raw_ratings.rating.astype('float')
ratings = raw_ratings[raw_ratings.userId.isin(user_list)]

In [ ]:
directors_popular_movies = tags[tags.movieId.isin(movies_list)]
directors_unique_users = list(tags_popular_movies.drop_duplicates('userId').userId)

In [ ]:
common_actors_df = tags_popular_movies.groupby(['tag']).count().sort_values('userId', ascending= False).copy().drop('movieId', axis= 1).reset_index()
common_tags_df = common_tags_df[common_tags_df.userId >= 20]
common_tags_df.to_csv('raw_data/common_tags.csv', index = False)

In [ ]:
# CELL 8

# Creating a new DF that contains the most common tags for each movie ("movieId"):

## This will create a new DF for each movie and will store this file since there is no easy storage method for this task
### Storage will be in the "data" folder under the "movie_tags" subfolder:

if os.path.exists('data/movie_directors/') != True: # Creating movie_tags subfolder
    os.mkdir('data/movie_directors/')

## Creating a copy of tags_df_mod and dropping userID:
tags_df_mod = pd.read_csv('data/tags.csv')

tags_df_no_user = tags_df_mod.copy().drop('userId', axis= 1)

## Obtaining a list of all movieId with tags:
### !!!! The set() function does not put the list in perfect order. Some of the IDs are out-of-place.
movieId_list = list(set(tags_df_no_user.movieId))

for movieId in movieId_list:
    df_select = tags_df_no_user[tags_df_no_user.movieId == movieId].copy().drop('movieId', axis= 1)

    df_select['COUNT'] = 1

    df_select_group = df_select.groupby(['tag']).count()

    df_select_group = df_select_group.sort_values(by=['COUNT'], ascending= False).reset_index()

    df_select_group.to_csv('data/movie_directors/' + str(movieId) + '.csv', index = False)

In [ ]:
# ratings_df_removed = pd.read_csv('data/ratings_df_last_liked_movie_removed.csv')

# user_list = list(set(ratings_df_removed.userId))

# like_dislike_directors = pd.DataFrame()

# for user in user_list:

#     temp_ratings_df = ratings_df_removed[ratings_df_removed.userId == user]
#     like_directors_df = pd.DataFrame()
#     dislike_directors_df = pd.DataFrame()

In [233]:
#     for index, row in temp_ratings_df.iterrows():  ## selecting favorite and least favorite directors for each user
#         try: ### This is to check if the directors exist
#             if row.rating >= 4: # Like
#                 temp_movie_df = pd.read_csv('data/movie_directors/{}.csv'.format(str(int(row.movieId)))) ## This oddly turns the movieId into a float, most likely to match the other data types in the selected series
#                 if len(like_directors_df) == 0:
#                     like_directors_df = temp_movie_df

#                 else:
#                     like_directors_df = pd.concat([like_directors_df, temp_movie_df], ignore_index= True)

#             else:
#                 temp_movie_df = pd.read_csv('data/movie_directors/{}.csv'.format(str(int(row.movieId))))

#                 if len(like_directors_df) == 0:
#                     dislike_directors_df = temp_movie_df

#                 else:
#                     dislike_directors_df = pd.concat([dislike_directors_df, temp_movie_df], ignore_index= True)
#         except Exception:
#             pass






In [ ]:
#     ## Counting all directors
#     try:  ### This is to check if the user has movies they've liked or disliked. Users who only have liked movies will be skipped (example: userId 173)
#         like_directors_list = list(like_directors_df.director)
#         dislike_directors_list = list(dislike_directors_df.director)
#     except Exception:
#         continue

In [ ]:
#     like_dict = {}
#     dislike_dict = {}

#     for director in like_directors_list:
#         like_dict[director] = like_directors_list.count(director) * -1  ### This is multiple by -1 to convert it to a negative numerical count for the sorting that will be done next

#     for director in dislike_directors_list:
#         dislike_dict[director] = dislike_directors_list.count(director) * -1

In [ ]:
#     like_directors_counted = sorted(like_dict, key= lambda director: like_dict[director])  ## Returns a list of the tags
#     dislike_directors_counted = sorted(dislike_dict, key= lambda director: dislike_dict[director])

#     ## Converting the tags to vectorized tags but only for the first 50 tags from the like and dislike tags counted lists
#     like_directors_vectorized = []
#     dislike_directors_vectorized = []

#     if len(like_directors_counted) < 50:  ## Checking to make sure there is 50 tags in the counted lists
#         num_like_directors = len(like_directors_counted)
#     else:
#         num_like_directors = 50

#     if len(dislike_directors_counted) < 50:
#         num_dislike_directors = len(like_directors_counted)
#     else:
#         num_dislike_directors = 50

#     for director in like_directors_counted[:num_like_directors]:
#         try:  ### The tag might not exist in the vectorized dictionary
#             director_vector = vectorized_dict[director]
#             like_directors_vectorized.append(director_vector)
#         except Exception:
#             pass

#     for director in dislike_directors_counted[:num_dislike_directors]:
#         try:
#             director_vector = vectorized_dict[director]
#             dislike_director_vectorized.append(director_vector)
#         except Exception:
#             pass

In [240]:
'''THINK ABOUT THIS ONE(IF NUMBER OF DIRECTORS IS APPROPRIATE)'''
#     if len(like_directors_vectorized) < 20 or len(dislike_directors_vectorized) < 20:
#         continue  ## If any of the two are not 20 directors in length, then the user will be skipped

'THINK ABOUT THIS ONE(IF NUMBER OF DIRECTORS IS APPROPRIATE)'

In [ ]:
'''THINK ABOUT THIS ONE(IF NUMBER OF DIRECTORS IS APPROPRIATE)'''
#     ## Obtaining the most liked and disliked directors, 20 director each, and adding it to like_dislike_directors:
#     like_dislike_dict = {}

#     like_dislike_dict['userId'] = user

#     for x in range(20):
#         like_dislike_dict['LIKE_' + str(x)] = like_directors_vectorized[x]
#         like_dislike_dict['DISLIKE_' + str(x)] = dislike_directors_vectorized[x]

#     concat_df = pd.DataFrame(like_dislike_dict, index=[0])

#     if len(like_dislike_directors) == 0:
#         like_dislike_directors = concat_df

#     else:
#         like_dislike_directors = pd.concat([like_dislike_directors,concat_df], ignore_index= True)

In [ ]:
# like_dislike_directors = like_dislike_directors.astype('int64')
# like_dislike_directors.to_csv('data/final/like_dislike_directors.csv', index = False)

## Actors

In [ ]:
# ratings_df_removed = pd.read_csv('data/ratings_df_last_liked_movie_removed.csv')

# user_list = list(set(ratings_df_removed.userId))

# like_dislike_actors = pd.DataFrame()

# for user in user_list:

#     temp_ratings_df = ratings_df_removed[ratings_df_removed.userId == user]
#     like_actors_df = pd.DataFrame()
#     dislike_actors_df = pd.DataFrame()

In [ ]:
#     for index, row in temp_ratings_df.iterrows():  ## selecting favorite and least favorite actors for each user
#         try:
#             if row.rating >= 4: # Like
#                 temp_movie_df = pd.read_csv('data/movie_actors/{}.csv'.format(str(int(row.movieId)))) ## This oddly turns the movieId into a float, most likely to match the other data types in the selected series
#                 if len(like_actors_df) == 0:
#                     like_actors_df = temp_movie_df

#                 else:
#                     like_actors_df = pd.concat([like_actors_df, temp_movie_df], ignore_index= True)

#             else:
#                 temp_movie_df = pd.read_csv('data/movie_actors/{}.csv'.format(str(int(row.movieId))))

#                 if len(like_actors_df) == 0:
#                     dislike_actors_df = temp_movie_df

#                 else:
#                     dislike_actors_df = pd.concat([dislike_actors_df, temp_movie_df], ignore_index= True)
#         except Exception:
#             pass


In [ ]:
#     ## Counting all actors
#     try:  ### This is to check if the user has movies they've liked or disliked. Users who only have liked movies will be skipped (example: userId 173)
#         like_actors_list = list(like_actors_df.actor)
#         dislike_actors_list = list(dislike_actors_df.actor)
#     except Exception:
#         continueactors

In [ ]:
#     like_dict = {}
#     dislike_dict = {}

#     for actor in like_actors_list:
#         like_dict[actor] = like_actors_list.count(actor) * -1  ### This is multiple by -1 to convert it to a negative numerical count for the sorting that will be done next

#     for actor in dislike_actors_list:
#         dislike_dict[actor] = dislike_actors_list.count(actor) * -1

In [ ]:
#     like_actors_counted = sorted(like_dict, key= lambda actor: like_dict[actor])  ## Returns a list of the tags
#     dislike_actors_counted = sorted(dislike_dict, key= lambda actor: dislike_dict[actor])

#     ## Converting the tags to vectorized tags but only for the first 50 tags from the like and dislike tags counted lists
#     like_actors_vectorized = []
#     dislike_actors_vectorized = []

#     if len(like_actors_counted) < 50:  ## Checking to make sure there is 50 tags in the counted lists
#         num_like_actors = len(like_actors_counted)
#     else:
#         num_like_actors = 50

#     if len(dislike_actors_counted) < 50:
#         num_dislike_actors = len(like_actors_counted)
#     else:
#         num_dislike_actors = 50

#     for actor in like_actors_counted[:num_like_actors]:
#         try:  ### The tag might not exist in the vectorized dictionary
#             actor_vector = vectorized_dict[actor]
#             like_actors_vectorized.append(actor_vector)
#         except Exception:
#             pass

#     for actor in dislike_actors_counted[:num_dislike_actors]:
#         try:
#             actor_vector = vectorized_dict[actor]
#             dislike_actors_vectorized.append(actor_vector)
#         except Exception:
#             pass

In [ ]:
#     if len(like_actors_vectorized) < 20 or len(dislike_actors_vectorized) < 20:
#         continue  ## If any of the two are not 20 tags in length, then the user will be skipped

In [ ]:
'''THINK ABOUT THIS ONE(IF NUMBER OF TAGS IS APPROPRIATE)'''
#     ## Obtaining the most liked and disliked actors, 20 tags each, and adding it to like_dislike_actors:
#     like_dislike_dict = {}

#     like_dislike_dict['userId'] = user

#     for x in range(20):
#         like_dislike_dict['LIKE_' + str(x)] = like_actors_vectorized[x]
#         like_dislike_dict['DISLIKE_' + str(x)] = dislike_actors_vectorized[x]

#     concat_df = pd.DataFrame(like_dislike_dict, index=[0])

#     if len(like_dislike_actors) == 0:
#         like_dislike_actors = concat_df

#     else:
#         like_dislike_actors = pd.concat([like_dislike_actors,concat_df], ignore_index= True)

In [ ]:
# like_dislike_actors = like_dislike_actors.astype('int64')
# like_dislike_actors.to_csv('data/final/like_dislike_actors.csv', index = False)

## Country of Origin

In [241]:
# ratings_df_removed = pd.read_csv('data/ratings_df_last_liked_movie_removed.csv')

# user_list = list(set(ratings_df_removed.userId))

# like_dislike_countries = pd.DataFrame()

# for user in user_list:

#     temp_ratings_df = ratings_df_removed[ratings_df_removed.userId == user]
#     like_countries_df = pd.DataFrame()
#     dislike_countries_df = pd.DataFrame()

In [ ]:
#     for index, row in temp_ratings_df.iterrows():  ## selecting favorite and least favorite countries for each user
#         try:
#             if row.rating >= 4: # Like
#                 temp_movie_df = pd.read_csv('data/movie_countries/{}.csv'.format(str(int(row.movieId)))) ## This oddly turns the movieId into a float, most likely to match the other data types in the selected series
#                 if len(like_countries_df) == 0:
#                     like_countries_df = temp_movie_df

#                 else:
#                     like_countries_df = pd.concat([like_countries_df, temp_movie_df], ignore_index= True)

#             else:
#                 temp_movie_df = pd.read_csv('data/movie_countries/{}.csv'.format(str(int(row.movieId))))

#                 if len(like_countries_df) == 0:
#                     dislike_countries_df = temp_movie_df

#                 else:
#                     dislike_countries_df = pd.concat([dislike_countries_df, temp_movie_df], ignore_index= True)
#         except Exception:
#             pass

In [ ]:
#     ## Counting all countries
#     try:  ### This is to check if the user has movies they've liked or disliked. Users who only have liked movies will be skipped (example: userId 173)
#         like_countries_list = list(like_countries_df.country)
#         dislike_countries_list = list(dislike_countries_df.country)
#     except Exception:
#         continue

In [ ]:
#     like_dict = {}
#     dislike_dict = {}

#     for country in like_countries_list:
#         like_dict[country] = like_countries_list.count(country) * -1  ### This is multiple by -1 to convert it to a negative numerical count for the sorting that will be done next

#     for country in dislike_countries_list:
#         dislike_dict[country] = dislike_countries_list.count(country) * -1

In [ ]:
#     like_countries_counted = sorted(like_dict, key= lambda country: like_dict[country])  ## Returns a list of the tags
#     dislike_countries_counted = sorted(dislike_dict, key= lambda country: dislike_dict[country])

#     ## Converting the tags to vectorized tags but only for the first 50 tags from the like and dislike tags counted lists
#     like_countries_vectorized = []
#     dislike_countries_vectorized = []

#     if len(like_countries_counted) < 50:  ## Checking to make sure there is 50 tags in the counted lists
#         num_like_countries = len(like_countries_counted)
#     else:
#         num_like_countries = 50

#     if len(dislike_countries_counted) < 50:
#         num_dislike_countries = len(like_countries_counted)
#     else:
#         num_dislike_countries = 50

#     for country in like_countries_counted[:num_like_countries]:
#         try:  ### The tag might not exist in the vectorized dictionary
#             country_vector = vectorized_dict[country]
#             like_countries_vectorized.append(country_vector)
#         except Exception:
#             pass

#     for country in dislike_countries_counted[:num_dislike_countries]:
#         try:
#             country_vector = vectorized_dict[country]
#             dislike_countries_vectorized.append(country_vector)
#         except Exception:
#             pass

In [ ]:
#     if len(like_countries_vectorized) < 20 or len(dislike_countries_vectorized) < 20:
#         continue  ## If any of the two are not 20 tags in length, then the user will be skipped

In [ ]:
'''THINK ABOUT THIS ONE(IF NUMBER OF TAGS IS APPROPRIATE)'''
#     ## Obtaining the most liked and disliked countries, 20 tags each, and adding it to like_dislike_countrie:
#     like_dislike_dict = {}

#     like_dislike_dict['userId'] = user

#     for x in range(20):
#         like_dislike_dict['LIKE_' + str(x)] = like_countries_vectorized[x]
#         like_dislike_dict['DISLIKE_' + str(x)] = dislike_countries_vectorized[x]

#     concat_df = pd.DataFrame(like_dislike_dict, index=[0])

#     if len(like_dislike_countries) == 0:
#         like_dislike_countries = concat_df

#     else:
#         like_dislike_countries = pd.concat([like_dislike_countries,concat_df], ignore_index= True)

In [ ]:
# like_dislike_countries = like_dislike_countries.astype('int64')
# like_dislike_countries.to_csv('data/final/like_dislike_countries.csv', index = False)